# MNL probability work

Sam Maurer, Sep 2018

This notebook is for feature development related to ChoiceModels [issue #26](https://github.com/UDST/choicemodels/issues/26)

### Set up an MNL model using fake data

In [1]:
import orca
import numpy as np
import pandas as pd

from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep

In [6]:
d1 = {'oid': np.arange(100), 
      'obsval': np.random.random(100),
      'choice': np.random.choice(np.arange(20), size=100)}

d2 = {'aid': np.arange(20), 
      'altval': np.random.random(20)}

obs = pd.DataFrame(d1).set_index('oid')
orca.add_table('obs', obs)

alts = pd.DataFrame(d2).set_index('aid')
orca.add_table('alts', alts)

In [7]:
mm.initialize()

No yaml files found in path 'configs'


In [8]:
m = LargeMultinomialLogitStep()
m.choosers = 'obs'
m.alternatives = 'alts'
m.choice_column = 'choice'
m.model_expression = 'obsval + altval - 1'

m.fit()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:            100
Model:         Multinomial Logit   Df Residuals:                 98
Method:       Maximum Likelihood   Df Model:                      2
Date:                 2018-09-11   Pseudo R-squ.:             0.000
Time:                      11:07   Pseudo R-bar-squ.:        -0.007
AIC:                     592.880   Log-Likelihood:         -294.440
BIC:                     598.090   LL-Null:                -294.444
             coef   std err         z     P>|z|   Conf. Int.
------------------------------------------------------------
obsval     0.0000     0.272     0.000     1.000             
altval    -0.0317     0.273    -0.116     0.908             


/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:233: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._df[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-mbp13/udst/choicemodels/choicemodels/mnl.py:236: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)


### Refactor MNL probability calculations

These are moving from a set of functions in `urbansim.urbanchoice.mnl` to a method of the `choicemodels.MultinomialLogitResults` class. It's easy to create a results class from a model expression and list of fitted coefficients, if a user needs to get probabilities extemporaneously.